<a href="https://colab.research.google.com/github/alifaisal-sadeq/skills-introduction-to-github/blob/main/Programming_Exercise_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install pyswarms if not already installed (run this once)
!pip install pyswarms

In [4]:
import numpy as np
import pyswarms as ps

c_i = np.array([30, 35, 40, 45, 50])
c_io = 1.5 * c_i
c_jp = np.array([25, 27, 29, 31, 33])
n_fulltime = len(c_i)
n_parttime = len(c_jp)
n_variables = n_fulltime * 2 + n_parttime

def objective_function(X):
    # X: shape (n_particles, n_variables)
    # Split X into x_i, x_io, x_jp
    x_i = X[:, :n_fulltime]
    x_io = X[:, n_fulltime:2*n_fulltime]
    x_jp = X[:, 2*n_fulltime:]

    cost_regular = np.sum(x_i * c_i, axis=1)
    cost_overtime = np.sum(x_io * c_io, axis=1)
    cost_parttime = np.sum(x_jp * c_jp, axis=1)
    total_cost = cost_regular + cost_overtime + cost_parttime


    penalty = 0
    penalty_factor = 1e6

    peak_hours = np.sum(x_i, axis=1) + np.sum(x_io, axis=1)
    peak_violation = np.maximum(4 - peak_hours, 0)
    penalty += penalty_factor * peak_violation

    nonpeak_hours = np.sum(x_jp, axis=1)
    nonpeak_violation = np.maximum(2 - nonpeak_hours, 0)
    penalty += penalty_factor * nonpeak_violation

    return total_cost + penalty

def pso_cost(X):
    return np.array([objective_function(X[i].reshape(1, -1)) for i in range(X.shape[0])]).flatten()

# PSO parameters
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
n_particles = 50
max_iterations = 100
lower_bound = np.zeros(n_variables)
upper_bound = np.ones(n_variables) * 8
bounds = (lower_bound, upper_bound)

#run PSO
optimizer = ps.single.GlobalBestPSO(
    n_particles=n_particles,
    dimensions=n_variables,
    options=options,
    bounds=bounds
)
best_cost, best_pos = optimizer.optimize(pso_cost, iters=max_iterations)

x_i_opt = best_pos[:n_fulltime]
x_io_opt = best_pos[n_fulltime:2*n_fulltime]
x_jp_opt = best_pos[2*n_fulltime:]

# results
print("\nOptimal Schedule:")
print("Full-time regular hours (x_i):", np.round(x_i_opt, 2))
print("Full-time overtime hours (x_io):", np.round(x_io_opt, 2))
print("Part-time hours (x_jp):", np.round(x_jp_opt, 2))
print("Total labor cost: $", np.round(best_cost, 2))

# Verify constraints
peak_total = np.sum(x_i_opt) + np.sum(x_io_opt)
nonpeak_total = np.sum(x_jp_opt)
print("Peak hours total (should be >= 4):", np.round(peak_total, 2))
print("Non-peak hours total (should be >= 2):", np.round(nonpeak_total, 2))

2025-04-04 13:29:32,450 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=762
2025-04-04 13:29:33,573 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 762.4005260267063, best pos: [2.50827209 0.03552902 1.3046761  0.19620643 0.43597198 3.42378345
 0.1669895  2.06591021 2.16025988 0.93793402 0.32860975 0.29569278
 0.30382752 0.380019   1.91966572]



Optimal Schedule:
Full-time regular hours (x_i): [2.51 0.04 1.3  0.2  0.44]
Full-time overtime hours (x_io): [3.42 0.17 2.07 2.16 0.94]
Part-time hours (x_jp): [0.33 0.3  0.3  0.38 1.92]
Total labor cost: $ 762.4
Peak hours total (should be >= 4): 13.24
Non-peak hours total (should be >= 2): 3.23
